Since no package is perfect and pytraj does not want to be perfect, we are trying to have a 'smooth talk' between `pytraj`
and other packages such as pysander (will be released in AmberTools15), ParmEd (package for editing Topology and many more),
or mdtraj and MDAnalysis (2 Python packages for MD analysis).

    * Note: (to be filled more)

## ParmEd

In [2]:
## ParmEd

import chemistry
from chemistry import load_file

pname = "../tests/data/Tc5b.top"

# load to ParmEd object
parm = load_file(pname)
print ("from ParmEd")
print (parm.__repr__())

# convert to pytraj's Topology object
# This is only pseudoparm, which means there is only Atom mask, residues, ... with no bonds. 
from pytraj import io
pseudo_top = io.load_pseudo_parm(parm)
print ("from pytraj") # no bonds info.
print (pseudo_top)

# we can load full Topology information to (behind the scene: pytraj would save and reload file)
full_parm = io.load_full_ParmEd(parm)
print (full_parm) # having bond info (will be added if anyone thinks this should be)

No module named 'pytz'
from ParmEd
<AmberParm 304 atoms; 20 residues; 310 bonds; parametrized>
from pytraj
Topology instance with 20 residues 304 atoms
Topology instance with 20 residues 304 atoms


In [8]:
# the point of using ParmEd object in pytraj is to post-process energy in simulation

# we can load new trajectory
traj = io.load("../tests/data/md1_prod.Tc5b.x", pname)
print (traj)

# to calculat energy, we need to use pysander (python wrapper of `sander` program in Amber). pysander will be released
# next few months in AmberTools15. stay tuned.

# pytraj has wrapper to call pysander internally
from pytraj.common_actions import calc_energies

# calculate bond, dihedral, ... and implicit solvent (igb=8) energies.
e_dict = calc_energies(parm, traj, igb=8)
print (e_dict.keys())

TrajReadOnly instance with 10 frames, 304 atoms/frame
           
dict_keys(['scf', 'hbond', 'polar', 'cmap', 'elec', 'surf', 'noe', 'dvdl', 'gb', 'angle_ub', 'constraint', 'bond', 'elec_14', 'emap', 'rism', 'les', 'amd_boost', 'dihedral', 'vdw', 'angle', 'imp', 'vdw_14', 'pb', 'disp', 'tot', 'ct'])


In [9]:
print (e_dict['bond'])

[69.75085151282191, 67.32323037339502, 54.356569176241145, 58.91231967248614, 80.72025558530078, 66.27135515789608, 81.91683307174073, 77.62477894319248, 74.49078120246152, 76.49473283118374]


## matraj

In [14]:
# pytraj can load and can be used (partially) in `mdtraj` package too

import mdtraj as md

# load mdtraj object
m_top = md.load_prmtop("../tests/data/Tc5b.top")
m_traj = md.load_mdcrd("../tests/data/md1_prod.Tc5b.x", m_top)
print (m_traj, m_traj.__class__)

<mdtraj.Trajectory with 10 frames, 304 atoms, 20 residues, without unitcells> <class 'mdtraj.core.trajectory.Trajectory'>


In [20]:
# let's load mdtraj object
from pytraj import io

pm_traj = io.load_mdtraj(m_traj)
print(pm_traj)
print (pm_traj.top)

FrameArray instance with 10 frames, 304 atoms/frame
           
Topology instance with 20 residues 304 atoms
[-1.64920008  1.24339998 -1.10179996]


In [21]:
# pytraj's object can be partially used in `mdtraj` too (as long as mdtraj use `xyz` ndarray to perform calculation

# `traj` is pytraj object in previous example

# use mdtraj to calculate rmsd to 1st frame, all atoms
arr0 = md.rmsd(traj, traj, 0) # used pytraj's traj object in mdtraj
print (arr0)

# calculate rmsd in pytraj
import numpy as np # for nicer printing
f0 = traj[0]
print (np.asarray([frame.rmsd(f0) for frame in traj])) # the results from pytraj and mdtraj are the same

[  0.          10.39640522  10.62932301  12.00149822  11.76797771
  11.38467979  11.92891216  11.82026386  11.63853741  12.42151833]
[  0.          10.39640617  10.62932356  12.00149779  11.7679772
  11.38468113  11.92891233  11.82026324  11.63853706  12.42151831]
